In [ ]:
# train script
# adapted from: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from PIL import Image
import ssl
from torch.cuda.amp import autocast, GradScaler
from torchvision.models import VisionTransformer

In [ ]:


#ssl._create_default_https_context = ssl._create_unverified_context

#if __name__ == '__main__':
scaler = GradScaler()
## cifar-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

#training set
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

#test set
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)
    
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# example images
dataiter = iter(trainloader)
images, labels = next(dataiter) # note: for pytorch versions (<1.14) use dataiter.next()

#TODO:
#mix up

im = Image.fromarray((torch.cat(images.split(1,0),3).squeeze()/2*255+.5*255).permute(1,2,0).numpy().astype('uint8'))
im.save("train_pt_images.jpg")
print('train_pt_images.jpg saved.')
print('Ground truth labels:' + ' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))


#training
## vision transformer 
net = VisionTransformer(image_size=32, patch_size=2, num_layers=6, num_heads=16,
                        hidden_dim=512, mlp_dim=2048, num_classes=len(classes)).cuda()

## loss and optimiser
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
results_train = []
results_test = []

## train
for epoch in range(20):  # loop over the dataset multiple times
    correct_train = 0
    total_train = 0
    running_loss = 0.0
    net.train()
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        #TODO: maybe romove autocast for the final one
        with autocast():
            outputs = net(inputs.cuda())
            #compute accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels.cuda()).sum().item()
            loss = criterion(outputs, labels.cuda())
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)  # Call scaler.step() instead of optimizer.step()
        scaler.update()  # Update the scal
        # loss.backward()
        # optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
        accuracy = 100 * correct_train / total_train
    print('Training done.')
    results_train.append(accuracy)
    print('Accuracy of the network on the train images: {}%'.format(accuracy))
    
    # evaluation on test
    net.eval()
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images.cuda())
            _, predicted = torch.max(outputs.data, 1)
            total_test += labels.size(0)
            correct_test += (predicted == labels.cuda()).sum().item()

    print('Accuracy of the network on the test images: {} %'.format(100 * correct_test / total_test))
# save trained model
torch.save(net.state_dict(), 'saved_model.pt')
print('Model saved.')


Files already downloaded and verified
Files already downloaded and verified
mixup.jpg saved.
Ground truth labels:plane   car truck horse  bird   dog  ship plane plane  ship horse   car truck   dog   dog   car
[1,   200] loss: 0.212
[1,   400] loss: 0.207
[1,   600] loss: 0.198
[1,   800] loss: 0.198
[1,  1000] loss: 0.197
[1,  1200] loss: 0.193
[1,  1400] loss: 0.192
[1,  1600] loss: 0.191
[1,  1800] loss: 0.188
[1,  2000] loss: 0.185
[1,  2200] loss: 0.187
[1,  2400] loss: 0.180
[1,  2600] loss: 0.180
[1,  2800] loss: 0.179
[1,  3000] loss: 0.180
Training done.
Epoch 1, Training accuracy: tensor([2.0820, 2.0340, 2.0480, 2.0280, 2.0220, 1.9760, 2.0020, 1.9860, 1.9860,
        1.9740, 1.9880, 1.9920, 2.0200, 1.9640, 2.0880, 2.0660],
       device='cuda:0')%
Epoch 1, Testing accuracy: tensor([0.3200, 0.5100, 0.4600, 0.4300, 0.5300, 0.4600, 0.4600, 0.3100, 0.5700,
        0.5800, 0.3800, 0.5300, 0.4500, 0.5300, 0.5300, 0.5200],
       device='cuda:0')%
[2,   200] loss: 0.174
[2,   400] loss: 0.179
[2,   600] loss: 0.175
[2,   800] loss: 0.174
[2,  1000] loss: 0.177
[2,  1200] loss: 0.174
[2,  1400] loss: 0.171
[2,  1600] loss: 0.170
[2,  1800] loss: 0.166
[2,  2000] loss: 0.171
[2,  2200] loss: 0.169
[2,  2400] loss: 0.170
[2,  2600] loss: 0.172
[2,  2800] loss: 0.166
[2,  3000] loss: 0.167
Training done.
Epoch 2, Training accuracy: tensor([2.5940, 2.5900, 2.5980, 2.5360, 2.6320, 2.5240, 2.5180, 2.6520, 2.4840,
        2.5180, 2.6100, 2.4840, 2.6220, 2.5440, 2.4640, 2.4800],
       device='cuda:0')%
Epoch 2, Testing accuracy: tensor([0.4100, 0.5400, 0.4900, 0.4700, 0.4400, 0.4600, 0.4900, 0.3700, 0.5000,
        0.7600, 0.4100, 0.6600, 0.4500, 0.5400, 0.6400, 0.5200],
       device='cuda:0')%
[3,   200] loss: 0.168
[3,   400] loss: 0.165
[3,   600] loss: 0.163
[3,   800] loss: 0.165
[3,  1000] loss: 0.165
[3,  1200] loss: 0.166
[3,  1400] loss: 0.164
[3,  1600] loss: 0.160
[3,  1800] loss: 0.162
[3,  2000] loss: 0.161
[3,  2200] loss: 0.164
[3,  2400] loss: 0.160
[3,  2600] loss: 0.160
[3,  2800] loss: 0.160
[3,  3000] loss: 0.159
Training done.
Epoch 3, Training accuracy: tensor([2.7820, 2.8600, 2.8240, 2.7140, 2.8260, 2.7880, 2.7860, 2.7820, 2.8240,
        2.7420, 2.8360, 2.7900, 2.8400, 2.7100, 2.8320, 2.8740],
       device='cuda:0')%
Epoch 3, Testing accuracy: tensor([0.3600, 0.5200, 0.4300, 0.4800, 0.3700, 0.3800, 0.4100, 0.4100, 0.5200,
        0.6300, 0.4800, 0.5400, 0.5500, 0.5200, 0.5200, 0.4700],
       device='cuda:0')%
[4,   200] loss: 0.161
[4,   400] loss: 0.157
[4,   600] loss: 0.158
[4,   800] loss: 0.158
[4,  1000] loss: 0.159
[4,  1200] loss: 0.157
[4,  1400] loss: 0.155
[4,  1600] loss: 0.160
[4,  1800] loss: 0.156
[4,  2000] loss: 0.158
[4,  2200] loss: 0.156
[4,  2400] loss: 0.155
[4,  2600] loss: 0.163
[4,  2800] loss: 0.154
[4,  3000] loss: 0.156
Training done.
Epoch 4, Training accuracy: tensor([2.9540, 2.9920, 2.9220, 3.0560, 2.9380, 3.0180, 3.0520, 2.9460, 2.9200,
        2.8980, 2.9820, 2.9260, 2.9940, 3.0660, 3.0140, 3.0120],
       device='cuda:0')%
Epoch 4, Testing accuracy: tensor([0.3700, 0.3100, 0.3600, 0.3400, 0.3000, 0.3900, 0.3200, 0.3300, 0.3900,
        0.6500, 0.4100, 0.5900, 0.4100, 0.4500, 0.4700, 0.4500],
       device='cuda:0')%
[5,   200] loss: 0.153
[5,   400] loss: 0.153
[5,   600] loss: 0.152
[5,   800] loss: 0.153
[5,  1000] loss: 0.152
[5,  1200] loss: 0.152
[5,  1400] loss: 0.152
[5,  1600] loss: 0.151
[5,  1800] loss: 0.148
[5,  2000] loss: 0.152
[5,  2200] loss: 0.154
[5,  2400] loss: 0.151
[5,  2600] loss: 0.151
[5,  2800] loss: 0.152
[5,  3000] loss: 0.150
Training done.
Epoch 5, Training accuracy: tensor([3.0860, 3.1880, 3.0140, 3.1160, 3.1440, 3.1900, 3.0640, 3.1900, 3.1680,
        3.1140, 3.1020, 3.0960, 3.0740, 3.1680, 3.0760, 3.0540],
       device='cuda:0')%
Epoch 5, Testing accuracy: tensor([0.4000, 0.4400, 0.4600, 0.4200, 0.3600, 0.3400, 0.3500, 0.3100, 0.5000,
        0.6800, 0.3900, 0.5100, 0.4300, 0.5100, 0.6100, 0.5300],
       device='cuda:0')%
[6,   200] loss: 0.148
[6,   400] loss: 0.149
[6,   600] loss: 0.146
[6,   800] loss: 0.146
[6,  1000] loss: 0.149
[6,  1200] loss: 0.148
[6,  1400] loss: 0.145
[6,  1600] loss: 0.149
[6,  1800] loss: 0.148
[6,  2000] loss: 0.151
[6,  2200] loss: 0.144
[6,  2400] loss: 0.146
[6,  2600] loss: 0.149
[6,  2800] loss: 0.146
[6,  3000] loss: 0.144
Training done.
Epoch 6, Training accuracy: tensor([3.1900, 3.2580, 3.1640, 3.1960, 3.2620, 3.2920, 3.2120, 3.1760, 3.3580,
        3.2940, 3.2200, 3.2320, 3.2860, 3.2640, 3.1960, 3.3620],
       device='cuda:0')%
Epoch 6, Testing accuracy: tensor([0.4300, 0.4500, 0.3900, 0.3900, 0.3700, 0.4000, 0.4400, 0.3200, 0.4600,
        0.6400, 0.4400, 0.5200, 0.5100, 0.4900, 0.5200, 0.4800],
       device='cuda:0')%
[7,   200] loss: 0.142
[7,   400] loss: 0.142
[7,   600] loss: 0.147
[7,   800] loss: 0.141
[7,  1000] loss: 0.143
[7,  1200] loss: 0.143
[7,  1400] loss: 0.145
[7,  1600] loss: 0.143
[7,  1800] loss: 0.141
[7,  2000] loss: 0.143
[7,  2200] loss: 0.142
[7,  2400] loss: 0.142
[7,  2600] loss: 0.143
[7,  2800] loss: 0.146
[7,  3000] loss: 0.141
Training done.
Epoch 7, Training accuracy: tensor([3.3480, 3.3280, 3.3740, 3.4100, 3.4020, 3.2760, 3.3100, 3.3260, 3.3900,
        3.2760, 3.3920, 3.3240, 3.3500, 3.4920, 3.3440, 3.2980],
       device='cuda:0')%
Epoch 7, Testing accuracy: tensor([0.3600, 0.5800, 0.4000, 0.4500, 0.3400, 0.3900, 0.3700, 0.4000, 0.5200,
        0.7100, 0.4500, 0.5000, 0.4900, 0.5100, 0.6600, 0.5700],
       device='cuda:0')%
[8,   200] loss: 0.138
[8,   400] loss: 0.138
[8,   600] loss: 0.138
[8,   800] loss: 0.141
[8,  1000] loss: 0.140
[8,  1200] loss: 0.141
[8,  1400] loss: 0.140
[8,  1600] loss: 0.136
[8,  1800] loss: 0.139
[8,  2000] loss: 0.136
[8,  2200] loss: 0.138
[8,  2400] loss: 0.140
[8,  2600] loss: 0.141
[8,  2800] loss: 0.137
[8,  3000] loss: 0.140
Training done.
Epoch 8, Training accuracy: tensor([3.4180, 3.4580, 3.5220, 3.4520, 3.3860, 3.5180, 3.3960, 3.4200, 3.4620,
        3.5060, 3.4700, 3.3620, 3.5200, 3.4960, 3.4920, 3.3920],
       device='cuda:0')%
Epoch 8, Testing accuracy: tensor([0.4100, 0.5800, 0.4200, 0.5400, 0.4200, 0.4000, 0.4000, 0.3100, 0.5600,
        0.6800, 0.5000, 0.4900, 0.4700, 0.5300, 0.5600, 0.4700],
       device='cuda:0')%
[9,   200] loss: 0.131
[9,   400] loss: 0.139
[9,   600] loss: 0.132
[9,   800] loss: 0.138
[9,  1000] loss: 0.135
[9,  1200] loss: 0.137
[9,  1400] loss: 0.137
[9,  1600] loss: 0.135
[9,  1800] loss: 0.134
[9,  2000] loss: 0.137
[9,  2200] loss: 0.136
[9,  2400] loss: 0.139
[9,  2600] loss: 0.135
[9,  2800] loss: 0.134
[9,  3000] loss: 0.134
Training done.
Epoch 9, Training accuracy: tensor([3.5580, 3.5600, 3.6480, 3.5720, 3.5160, 3.5480, 3.5940, 3.5320, 3.6200,
        3.6300, 3.4720, 3.5320, 3.6180, 3.4700, 3.5080, 3.6740],
       device='cuda:0')%
Epoch 9, Testing accuracy: tensor([0.3700, 0.4500, 0.4200, 0.4500, 0.3400, 0.3800, 0.3300, 0.3900, 0.3900,
        0.6200, 0.4100, 0.4800, 0.5200, 0.4800, 0.5000, 0.4000],
       device='cuda:0')%
[10,   200] loss: 0.134
[10,   400] loss: 0.130
[10,   600] loss: 0.130
[10,   800] loss: 0.131
[10,  1000] loss: 0.129
[10,  1200] loss: 0.130
[10,  1400] loss: 0.135
[10,  1600] loss: 0.134
[10,  1800] loss: 0.132
[10,  2000] loss: 0.133
[10,  2200] loss: 0.130
[10,  2400] loss: 0.129
[10,  2600] loss: 0.134
[10,  2800] loss: 0.134
[10,  3000] loss: 0.133
Training done.
Epoch 10, Training accuracy: tensor([3.5660, 3.6900, 3.6860, 3.6120, 3.5740, 3.6020, 3.7520, 3.6860, 3.6480,
        3.6180, 3.7240, 3.5560, 3.6120, 3.6200, 3.6220, 3.5700],
       device='cuda:0')%
Epoch 10, Testing accuracy: tensor([0.4400, 0.5600, 0.3800, 0.4300, 0.4700, 0.3600, 0.3600, 0.3800, 0.4700,
        0.7100, 0.4300, 0.4600, 0.5200, 0.5900, 0.6000, 0.6200],
       device='cuda:0')%
[11,   200] loss: 0.127
[11,   400] loss: 0.125
[11,   600] loss: 0.127
[11,   800] loss: 0.134
[11,  1000] loss: 0.128
[11,  1200] loss: 0.129
[11,  1400] loss: 0.132
[11,  1600] loss: 0.128
[11,  1800] loss: 0.130
[11,  2000] loss: 0.129
[11,  2200] loss: 0.128
[11,  2400] loss: 0.127
[11,  2600] loss: 0.128
[11,  2800] loss: 0.127
[11,  3000] loss: 0.129
Training done.
Epoch 11, Training accuracy: tensor([3.7080, 3.7440, 3.7760, 3.6140, 3.7340, 3.7340, 3.8400, 3.7520, 3.8860,
        3.6900, 3.7240, 3.7780, 3.7860, 3.6420, 3.6780, 3.7480],
       device='cuda:0')%
Epoch 11, Testing accuracy: tensor([0.4100, 0.4000, 0.3400, 0.3900, 0.4000, 0.3400, 0.4200, 0.3700, 0.4600,
        0.6800, 0.4000, 0.5800, 0.5600, 0.4900, 0.5800, 0.5200],
       device='cuda:0')%
[12,   200] loss: 0.121
[12,   400] loss: 0.122
[12,   600] loss: 0.125
[12,   800] loss: 0.125
[12,  1000] loss: 0.122
[12,  1200] loss: 0.121
[12,  1400] loss: 0.123
[12,  1600] loss: 0.125
[12,  1800] loss: 0.126
[12,  2000] loss: 0.124
[12,  2200] loss: 0.126
[12,  2400] loss: 0.123
[12,  2600] loss: 0.123
[12,  2800] loss: 0.124
[12,  3000] loss: 0.126
Training done.
Epoch 12, Training accuracy: tensor([3.8660, 3.8940, 3.8640, 3.7640, 3.8000, 3.8320, 3.7720, 3.8580, 3.8960,
        3.8020, 3.8320, 3.8580, 3.7900, 3.8500, 3.8800, 3.8580],
       device='cuda:0')%
Epoch 12, Testing accuracy: tensor([0.4400, 0.3900, 0.3500, 0.4500, 0.4000, 0.3500, 0.3800, 0.3500, 0.4300,
        0.6600, 0.3700, 0.5300, 0.5900, 0.4600, 0.6100, 0.4500],
       device='cuda:0')%
[13,   200] loss: 0.121
[13,   400] loss: 0.121
[13,   600] loss: 0.119
[13,   800] loss: 0.119
[13,  1000] loss: 0.121
[13,  1200] loss: 0.119
[13,  1400] loss: 0.120
[13,  1600] loss: 0.122
[13,  1800] loss: 0.122
[13,  2000] loss: 0.123
[13,  2200] loss: 0.122
[13,  2400] loss: 0.120
[13,  2600] loss: 0.123
[13,  2800] loss: 0.123
[13,  3000] loss: 0.118
Training done.
Epoch 13, Training accuracy: tensor([3.8860, 3.8840, 3.9000, 3.8740, 3.9020, 3.8880, 3.9540, 3.9700, 3.9620,
        4.0120, 3.8960, 4.0220, 3.8920, 3.9140, 3.9240, 3.9900],
       device='cuda:0')%
Epoch 13, Testing accuracy: tensor([0.4400, 0.5500, 0.4000, 0.4900, 0.4400, 0.3900, 0.3800, 0.4300, 0.4600,
        0.6200, 0.4300, 0.5900, 0.4500, 0.4300, 0.6100, 0.5400],
       device='cuda:0')%
[14,   200] loss: 0.116
[14,   400] loss: 0.117
[14,   600] loss: 0.115
[14,   800] loss: 0.119
[14,  1000] loss: 0.120
[14,  1200] loss: 0.118
[14,  1400] loss: 0.116
[14,  1600] loss: 0.118
[14,  1800] loss: 0.114
[14,  2000] loss: 0.118
[14,  2200] loss: 0.117
[14,  2400] loss: 0.117
[14,  2600] loss: 0.117
[14,  2800] loss: 0.115
[14,  3000] loss: 0.118
Training done.
Epoch 14, Training accuracy: tensor([4.0500, 4.0420, 4.0180, 4.1160, 4.1680, 4.0500, 3.9580, 3.9840, 4.1360,
        3.9620, 4.0560, 3.9760, 3.9820, 4.0100, 4.1080, 4.0220],
       device='cuda:0')%
Epoch 14, Testing accuracy: tensor([0.4300, 0.6500, 0.4000, 0.4500, 0.4600, 0.4100, 0.3000, 0.4100, 0.5100,
        0.6500, 0.5200, 0.5300, 0.5300, 0.5300, 0.5400, 0.5100],
       device='cuda:0')%
[15,   200] loss: 0.112
[15,   400] loss: 0.113
[15,   600] loss: 0.114
[15,   800] loss: 0.115
[15,  1000] loss: 0.113
[15,  1200] loss: 0.115
[15,  1400] loss: 0.115
[15,  1600] loss: 0.115
[15,  1800] loss: 0.111
[15,  2000] loss: 0.113
[15,  2200] loss: 0.118
[15,  2400] loss: 0.115
[15,  2600] loss: 0.118
[15,  2800] loss: 0.115
[15,  3000] loss: 0.116
Training done.
Epoch 15, Training accuracy: tensor([4.0780, 4.1700, 4.0840, 4.0540, 4.0960, 4.0260, 4.0540, 4.0600, 4.1420,
        4.0480, 4.1640, 4.1320, 4.1220, 4.1040, 4.0660, 4.1140],
       device='cuda:0')%
Epoch 15, Testing accuracy: tensor([0.4500, 0.4500, 0.4100, 0.5000, 0.4400, 0.3800, 0.3600, 0.3600, 0.4200,
        0.5700, 0.4600, 0.5100, 0.4500, 0.5300, 0.5900, 0.4700],
       device='cuda:0')%
[16,   200] loss: 0.109
[16,   400] loss: 0.108
[16,   600] loss: 0.115
[16,   800] loss: 0.107
[16,  1000] loss: 0.112
[16,  1200] loss: 0.113
[16,  1400] loss: 0.114
[16,  1600] loss: 0.112
[16,  1800] loss: 0.113
[16,  2000] loss: 0.112
[16,  2200] loss: 0.114
[16,  2400] loss: 0.112
[16,  2600] loss: 0.112
[16,  2800] loss: 0.112
[16,  3000] loss: 0.115
Training done.
Epoch 16, Training accuracy: tensor([4.1920, 4.0960, 4.2400, 4.1860, 4.1980, 4.1640, 4.1640, 4.1860, 4.2080,
        4.0860, 4.2180, 4.1400, 4.1900, 4.2040, 4.1860, 4.1000],
       device='cuda:0')%
Epoch 16, Testing accuracy: tensor([0.5000, 0.5300, 0.3900, 0.4400, 0.4600, 0.3500, 0.3700, 0.4200, 0.4700,
        0.6700, 0.5000, 0.5500, 0.5600, 0.4900, 0.6200, 0.5400],
       device='cuda:0')%
[17,   200] loss: 0.108
[17,   400] loss: 0.105
[17,   600] loss: 0.106
[17,   800] loss: 0.108
[17,  1000] loss: 0.107
[17,  1200] loss: 0.107
[17,  1400] loss: 0.110
[17,  1600] loss: 0.112
[17,  1800] loss: 0.108
[17,  2000] loss: 0.108
[17,  2200] loss: 0.109
[17,  2400] loss: 0.109
[17,  2600] loss: 0.110
[17,  2800] loss: 0.107
[17,  3000] loss: 0.110
Training done.
Epoch 17, Training accuracy: tensor([4.1760, 4.2100, 4.2880, 4.3580, 4.3320, 4.2780, 4.2020, 4.2600, 4.2800,
        4.2820, 4.2820, 4.2580, 4.3580, 4.2120, 4.2060, 4.2400],
       device='cuda:0')%
Epoch 17, Testing accuracy: tensor([0.4600, 0.4800, 0.3400, 0.4800, 0.4100, 0.3300, 0.3700, 0.3400, 0.5200,
        0.5900, 0.4600, 0.5600, 0.5500, 0.4900, 0.5400, 0.4900],
       device='cuda:0')%
[18,   200] loss: 0.102
[18,   400] loss: 0.104
[18,   600] loss: 0.105
[18,   800] loss: 0.103
[18,  1000] loss: 0.103
[18,  1200] loss: 0.108
[18,  1400] loss: 0.104
[18,  1600] loss: 0.103
[18,  1800] loss: 0.104
[18,  2000] loss: 0.104
[18,  2200] loss: 0.107
[18,  2400] loss: 0.104
[18,  2600] loss: 0.105
[18,  2800] loss: 0.106
[18,  3000] loss: 0.110
Training done.
Epoch 18, Training accuracy: tensor([4.3640, 4.2740, 4.3000, 4.3000, 4.3980, 4.3660, 4.3040, 4.3900, 4.2920,
        4.3760, 4.3120, 4.3640, 4.4060, 4.4160, 4.4080, 4.3120],
       device='cuda:0')%
Epoch 18, Testing accuracy: tensor([0.4500, 0.4800, 0.4300, 0.4800, 0.4000, 0.3400, 0.2900, 0.3400, 0.5000,
        0.5900, 0.4500, 0.4900, 0.4700, 0.5200, 0.5200, 0.4400],
       device='cuda:0')%
[19,   200] loss: 0.099
[19,   400] loss: 0.099
[19,   600] loss: 0.101
[19,   800] loss: 0.099
[19,  1000] loss: 0.103
[19,  1200] loss: 0.103
[19,  1400] loss: 0.102
[19,  1600] loss: 0.101
[19,  1800] loss: 0.103
[19,  2000] loss: 0.100
[19,  2200] loss: 0.101
[19,  2400] loss: 0.100
[19,  2600] loss: 0.102
[19,  2800] loss: 0.105
[19,  3000] loss: 0.105
Training done.
Epoch 19, Training accuracy: tensor([4.4500, 4.3720, 4.4560, 4.4860, 4.3520, 4.3540, 4.3920, 4.4400, 4.4620,
        4.4440, 4.4880, 4.4100, 4.3640, 4.3940, 4.4780, 4.5260],
       device='cuda:0')%
Epoch 19, Testing accuracy: tensor([0.4700, 0.4900, 0.4300, 0.4400, 0.4200, 0.4400, 0.3500, 0.3400, 0.3900,
        0.5900, 0.4200, 0.5300, 0.4800, 0.5600, 0.6000, 0.5700],
       device='cuda:0')%
[20,   200] loss: 0.092
[20,   400] loss: 0.095
[20,   600] loss: 0.096
[20,   800] loss: 0.095
[20,  1000] loss: 0.101
[20,  1200] loss: 0.098
[20,  1400] loss: 0.095
[20,  1600] loss: 0.099
[20,  1800] loss: 0.102
[20,  2000] loss: 0.099
[20,  2200] loss: 0.101
[20,  2400] loss: 0.096
[20,  2600] loss: 0.098
[20,  2800] loss: 0.098
[20,  3000] loss: 0.101
Training done.
Epoch 20, Training accuracy: tensor([4.4580, 4.4660, 4.4780, 4.6260, 4.5880, 4.5760, 4.6380, 4.5780, 4.4800,
        4.5440, 4.5060, 4.5840, 4.4720, 4.6200, 4.5740, 4.5200],
       device='cuda:0')%
Epoch 20, Testing accuracy: tensor([0.4300, 0.5100, 0.4200, 0.4900, 0.4400, 0.3900, 0.3800, 0.4200, 0.4500,
        0.6200, 0.4500, 0.5400, 0.5100, 0.4700, 0.5700, 0.4800],